In [5]:
import cv2
import numpy as np
import PySimpleGUI as sg
from PIL import Image, ImageTk
import os

In [6]:

# 画像を読み込む関数
def load_image(file_path):
    image = cv2.imread(file_path)
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

def get_txt(input_folder_path,output_folder_path):

    # valフォルダに含まれるすべてのPNGファイルの名前を取得
    png_files = [f for f in os.listdir(input_folder_path) if f.endswith(".png")]

    # 各PNGファイルに対応する.txtファイルを作成
    for png_file in png_files:
        # ファイル名の拡張子を除去
        file_name = os.path.splitext(png_file)[0]
        # .txtファイルのパスを作成
        txt_file_path = os.path.join(output_folder_path, f"{file_name}.txt")
        # 空の.txtファイルを作成
        with open(txt_file_path, 'w') as txt_file:
            pass

# RGB各チャネルを調整する関数
def adjust_rgb_channels(image, scale_r, scale_g, scale_b, offset_r, offset_g, offset_b):
    R, G, B = cv2.split(image)
    R = R * scale_r + offset_r
    G = G * scale_g + offset_g
    B = B * scale_b + offset_b
    # 値をクリップして範囲内に収める
    R = np.clip(R, 0, 255).astype(np.uint8)
    G = np.clip(G, 0, 255).astype(np.uint8)
    B = np.clip(B, 0, 255).astype(np.uint8)
    return cv2.merge((R, G, B))

# PySimpleGUIで画像を表示可能な形式に変換
def image_to_data(image):
    img = Image.fromarray(image)
    return ImageTk.PhotoImage(image=img)

# GUIレイアウト
layout = [
    [sg.Button("基準画像を選択", key='-SELECT-BASE-'), sg.Button("保存フォルダを指定", key='-SELECT-SAVE-FOLDER-'), sg.Button("終了", key='-EXIT-'),sg.Button("適用して保存", key='-APPLY-SAVE-')],
    [sg.Text("赤チャネル (Red Scale):"), sg.Slider(range=(0.1, 3.0), resolution=0.1, orientation='h', size=(40, 15), key='-SCALE_RED-', default_value=1.0)],
    [sg.Text("赤チャネル (Red Offset):"), sg.Slider(range=(-100, 100), resolution=1, orientation='h', size=(40, 15), key='-OFFSET_RED-', default_value=0)],
    [sg.Text("緑チャネル (Green Scale):"), sg.Slider(range=(0.1, 3.0), resolution=0.1, orientation='h', size=(40, 15), key='-SCALE_GREEN-', default_value=1.0)],
    [sg.Text("緑チャネル (Green Offset):"), sg.Slider(range=(-100, 100), resolution=1, orientation='h', size=(40, 15), key='-OFFSET_GREEN-', default_value=0)],
    [sg.Text("青チャネル (Blue Scale):"), sg.Slider(range=(0.1, 3.0), resolution=0.1, orientation='h', size=(40, 15), key='-SCALE_BLUE-', default_value=1.0)],
    [sg.Text("青チャネル (Blue Offset):"), sg.Slider(range=(-100, 100), resolution=1, orientation='h', size=(40, 15), key='-OFFSET_BLUE-', default_value=0)],
    [sg.Image(key='-IMAGE-')],
    
]

# ウィンドウを作成
window = sg.Window("RGB画像調整ツール", layout, resizable=True)

# イベントループ
base_image_path = None
base_image = None
adjusted_base_image = None
output_folder = None

while True:
    event, values = window.read(timeout=100)
    if event in (sg.WINDOW_CLOSED, '-EXIT-'):
        break
    
    # 基準画像を選択
    if event == '-SELECT-BASE-':
        base_image_path = sg.popup_get_file("基準画像を選択してください", file_types=(("画像ファイル", "*.png;*.jpg;*.jpeg")),initial_folder="Dataset")
        if base_image_path:
            base_image = load_image(base_image_path)
            adjusted_base_image = base_image.copy()
            window['-IMAGE-'].update(data=image_to_data(base_image))
    
    # 保存フォルダを選択
    if event == '-SELECT-SAVE-FOLDER-':
        output_folder = sg.popup_get_folder("保存先フォルダを選択してください",initial_folder="Dataset")
        if output_folder:
            sg.popup(f"保存先フォルダ: {output_folder}")
    
    # 基準画像のリアルタイム調整
    if base_image is not None:
        scale_r = values['-SCALE_RED-']
        offset_r = values['-OFFSET_RED-']
        scale_g = values['-SCALE_GREEN-']
        offset_g = values['-OFFSET_GREEN-']
        scale_b = values['-SCALE_BLUE-']
        offset_b = values['-OFFSET_BLUE-']
        
        adjusted_base_image = adjust_rgb_channels(base_image, scale_r, scale_g, scale_b, offset_r, offset_g, offset_b)
        window['-IMAGE-'].update(data=image_to_data(adjusted_base_image))
    
    # 全画像に適用して保存
    if event == '-APPLY-SAVE-':
        text = sg.popup_get_text("train or val?")
        if not base_image_path or not output_folder:
            sg.popup("基準画像または保存先フォルダが選択されていません！")
            continue
        
        input_folder = os.path.dirname(base_image_path)
        valid_extensions = (".png", ".jpg", ".jpeg")
        image_files = [f for f in os.listdir(input_folder) if f.lower().endswith(valid_extensions)]
        
        if not image_files:
            sg.popup("画像ファイルが見つかりません！")
            continue
        
        for image_file in image_files:
            file_path = os.path.join(input_folder, image_file)
            image = load_image(file_path)
            adjusted_image = adjust_rgb_channels(image, scale_r, scale_g, scale_b, offset_r, offset_g, offset_b)
            save_path = os.path.join(output_folder, f"{text}_adjusted_{image_file}")
            cv2.imwrite(save_path, cv2.cvtColor(adjusted_image, cv2.COLOR_RGB2BGR))
        
        get_txt("Dataset/original_process/train/image/processed_image", "Dataset/original_process/train/label/processed_label")
        get_txt("Dataset/val/images", "Dataset/val/labels")

        sg.popup(f"{len(image_files)} 個の画像を保存しました！")

window.close()


/Users/oikawakazutaka/Desktop/画像解析/.venv/lib/python3.12/site-packages/PySimpleGUI/PySimpleGUI.py:4601: UserWarning: file_types parameter not correctly specified. This parameter is a LIST of TUPLES. You have passed (str,str) rather than ((str, str),). Fixing it for you this time.
changing ('画像ファイル', '*.png;*.jpg;*.jpeg') to (('画像ファイル', '*.png;*.jpg;*.jpeg'),)
Please correct your code
  warnings.warn('file_types parameter not correctly specified. This parameter is a LIST of TUPLES. You have passed (str,str) rather than ((str, str),). Fixing it for you this time.\nchanging {} to {}\nPlease correct your code'.format(file_types, ((file_types[0], file_types[1]),)), UserWarning)
/Users/oikawakazutaka/Desktop/画像解析/.venv/lib/python3.12/site-packages/PySimpleGUI/PySimpleGUI.py:4601: UserWarning: file_types parameter not correctly specified. This parameter is a LIST of TUPLES. You have passed (str,str) rather than ((str, str),). Fixing it for you this time.
changing ('画像ファイル', '*.png;*.jpg;*.jpeg